In [1]:
from dotenv import load_dotenv
from requests import post, get
import os
import base64
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
from pprint import pprint
import pandas as pd

Api Keys & Env Variables

In [2]:
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

os.environ["SPOTIPY_CLIENT_ID"] = client_id
os.environ["SPOTIPY_CLIENT_SECRET"] = client_secret


In [3]:
client_id, client_secret

('3f1b02178e8446898db01fed44c33a5d', '815c09a17aa84a51ab099ea7dc58dc06')

Manual Spotify API requests (not used)

In [4]:
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type" : "client_credentials"}

    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token

In [5]:
def get_auth_header(token):
    return { "Authorization" : "Bearer " + token }

In [6]:
def search_for_track(token, query):
    url = 'https://api.spotify.com/v1/search'
    headers = get_auth_header(token)
    # query = f"?q={query}&type=track&limit=50"
    query = '?q=track:"praise god" year:2012-2019&type=track&limit=1&market=US'

    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)
    print(json_result)



In [7]:
def get_artist(token, artist_id):
    url = 'https://api.spotify.com/v1/artists/'
    headers = get_auth_header(token)
    # query = f"?q={query}&type=track&limit=50"
    query = artist_id

    query_url = url + query
    result = get(query_url, headers=headers)
    print(result)
    json_result = json.loads(result.content)
    print(json_result)


In [8]:
token = get_token()

In [9]:
result = get_artist(token, "14ERYNp2qiNhHlXxkRr05M")

<Response [429]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Random Song Search Helper Functions

In [15]:

def get_random_search():
    # A list of all characters that can be chosen.
    characters = 'abcdefghijklmnopqrstuvwxyz'
  
    # Gets a random character from the characters string.
    random_character = random.choice(characters)
    random_search = ''

    # Places the wildcard character at the beginning, or both beginning and end, randomly.
    if random.randint(0, 1) == 0:
        random_search = random_character + '%'
    else:
        random_search = '%' + random_character + '%'

    return random_search


Setup Auth & Auto-refresh token

In [13]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [36]:
album = sp.search(
    q='album:"graduation"',
    limit=1,
    offset=0,
    type='album',
    market='US'
)

Random Song Function (spotipy, spotify api)

In [25]:
def get_random_song() -> list:
    q = get_random_search()
    offset = random.randint(0, 999)
    year = str(2016 + random.randint(0, 5))

    # get bottom 10% popularity album using 'hipster' tag
    album = sp.search(
        q='album:"' + q + '" year:' + year + " tag:hipster",
        limit=1,
        offset=offset,
        type='album',
        market='US'
    )
    artist = album['albums']['items'][0]['artists'][0]['name']
    
    # get genre of album artist
    genres = sp.artist(album['albums']['items'][0]['artists'][0]['id'])['genres']

    # Now get a random song from the album
    album_id = album['albums']['items'][0]['id']
    total_tracks = album['albums']['items'][0]['total_tracks']
    track = sp.album_tracks(
        album_id=album_id,
        limit=1,
        offset=random.randint(0, total_tracks - 1),
        market='US'    
    )
    song = track['items'][0]['name']

    return (genres, artist, song)



In [26]:
genres, artist, song = get_random_song()


In [17]:
# genres, artist, song = get_random_song()
q = get_random_search()
offset = random.randint(0, 999)
year = str(2016 + random.randint(0, 5))

album = sp.search(
        q='album:"' + q + '" year:' + year + " tag:hipster",
        limit=1,
        offset=offset,
        type='album',
        market='US'
    )
artist = album['albums']['items'][0]['artists'][0]['name']

    

In [18]:
artist_id = album['albums']['items'][0]['artists'][0]['id']
print(artist_id)

0LyfQWJT6nXafLPZqxe9Of


In [19]:
genres = sp.artist(artist_id)

In [22]:
genres

[]

In [21]:
genres = sp.artist(album['albums']['items'][0]['artists'][0]['id'])['genres']


Save 3000 songs to CSV using Pandas (since there are ~3000 billboard songs)

In [27]:
song_limit = 3000

relative_csv_path = "../data/random_2016-2021_song_artist_genres.csv"
df = pd.read_csv(relative_csv_path)
assert list(df.columns) == ['song', 'artist', 'genres']

i = 0
while df.shape[0] < song_limit:
    try:
        genres, artist, song = get_random_song()
        if genres:
            row = pd.Series(data=[song, artist, genres], index=df.columns)
            df = pd.concat([df, row.to_frame().T], axis=0, ignore_index=True)
            df.to_csv(relative_csv_path, index=False)
    except Exception as e:
        print("ERROR: ", e)
        # break
    i+=1
    print(i, df.shape[0])


1 718
2 719
3 720
4 720
5 721
6 721
7 722
8 722
9 722
10 723
11 724
12 725
13 725
14 726
15 726
16 726
17 727
18 728
19 729
20 730
21 731
22 731
23 732
24 732
25 733
26 733
27 734
28 734
29 734
30 734
31 735
32 735
33 735
34 735
35 736
36 736
37 736
38 737
39 737
40 737
41 737
42 737
43 738
44 738
45 738
46 738
47 739
48 739
49 739
50 739
51 739
52 739
53 739
54 740
55 740
56 740
57 740
58 740
59 740
60 741
61 741
62 741
63 742
64 742
65 743
66 743
67 743
68 743
69 743
70 743
71 743
72 743
73 744
74 744
75 745
76 746
77 747
78 747
79 747
80 747
81 748
82 749
83 750
84 750
85 750
86 750
87 751
88 751
89 751
90 751
91 752
92 752
93 753
94 753
95 753
96 753
97 754
98 754
99 754
100 755
101 756
102 756
103 757
104 757
105 757
106 757
107 757
108 757
109 758
110 758
111 758
112 758
113 759
114 760
115 760
116 761
117 761
118 761
119 762
120 762
121 763
122 763
123 763
124 764
125 765
126 765
127 765
128 765
129 765
130 766
131 766
132 767
133 768
134 769
135 770
136 771
137 771
138 771
139 